# Tools for a Customer Outreach Campaign


In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [2]:
from crewai import Agent, Task, Crew

In [ ]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

os.environ["OPENAI_API_KEY"] = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4.1-nano'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## Creating Agents

In [4]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [5]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [6]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [7]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [14]:
from crewai.tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [15]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [16]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [17]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [18]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [20]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=False,
	memory=True
)

w:\CrewAI\L2\venv\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [24]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

# Agent: Sales Representative
## Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


2025-05-26 12:59:15,074 - 2648 - serper_dev_tool.py-serper_dev_tool:179 - ERROR: Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","statusCode":403}'
2025-05-26 12:59:15,353 - 2648 - serper_dev_tool.py-serper_dev_tool:179 - ERROR: Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","statusCode":403}'
2025-05-26 12:59:15,652 - 2648 - serper_dev_tool.py-serper_dev_tool:179 - ERROR: Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","statusCode":403}'
2025-05-26 12:59:15,962 - 2648 - serper_dev_tool.py-serper_dev_tool:179 - ERROR: Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","status

 

I encountered an error while trying to use the tool. This was the error: 403 Client Error: Forbidden for url: https://google.serper.dev/search.
 Tool Search the internet with Serper accepts these inputs: Tool Name: Search the internet with Serper
Tool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the internet', 'type': 'str'}}
Tool Description: A tool that can be used to search the internet with a search_query. Supports different search types: 'search' (default), 'news'



# Agent: Sales Representative
## Thought: To start the process of gathering comprehensive information about DeepLearningAI, I need to use the internet search tool to find key decision-makers, recent business developments, and potential needs.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"DeepLearningAI company profile key decision-makers business developments\"}"
## Tool Output: 

I encountered an error while trying to use the t

w:\CrewAI\L2\venv\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
w:\CrewAI\L2\venv\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


# Agent: Lead Sales Representative
## Task: Using the insights gathered from the lead profiling report on DeepLearningAI, craft a personalized outreach campaign aimed at Andrew Ng, the CEO of DeepLearningAI. The campaign should address their recent product launch and how our solutions can support their goals. Your communication must resonate with DeepLearningAI's company culture and values, demonstrating a deep understanding of their business and needs.
Don't make assumptions and only use information you absolutely sure about.


# Agent: Lead Sales Representative
## Final Answer: 
Subject: Unlock New Horizons in AI Education with CrewAI’s Tailored Solutions

Dear Andrew,

I hope this message finds you well. My name is [Your Name], and I lead the sales team at CrewAI, where we are dedicated to powering enterprises with cutting-edge AI solutions. Given DeepLearningAI’s recent strides in the AI education sector, I wanted to take this opportunity to congratulate you on your innovative prod

w:\CrewAI\L2\venv\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
w:\CrewAI\L2\venv\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


- Display the final result as Markdown.

In [26]:
from IPython.display import Markdown

Markdown(str(result))

Subject: Unlock New Horizons in AI Education with CrewAI’s Tailored Solutions

Dear Andrew,

I hope this message finds you well. My name is [Your Name], and I lead the sales team at CrewAI, where we are dedicated to powering enterprises with cutting-edge AI solutions. Given DeepLearningAI’s recent strides in the AI education sector, I wanted to take this opportunity to congratulate you on your innovative product launches this year.

At CrewAI, we are impressed by DeepLearningAI's commitment to making AI education accessible and comprehensive, aligning closely with our goal to enhance learning through technology. Our advanced AI-driven tools are designed to assist educational platforms like yours to engage more effectively with learners, streamline content delivery, and unlock in-depth insights into learning patterns.

We believe our solutions could seamlessly integrate with your platform, amplifying your capabilities to deliver personalized learning experiences. For instance, our AI analytics system could provide your team with detailed metrics on course effectiveness and learner engagement, making it easier to tailor content dynamically and improve educational outcomes.

I am keen to discuss how CrewAI can support DeepLearningAI in achieving its mission to democratize AI knowledge. Could we schedule a call next week to explore potential collaborations? I am confident there is much to be gained from a partnership between our companies.

Thank you for considering this opportunity. I look forward to your thoughts and hope to speak soon.

Warm regards,

[Your Name] [Your Position] CrewAI [Contact Information]